In [33]:
import json

with open("./all/openings.json", "r") as json_file:
    openings: list[dict[str]] = json.load(json_file)

In [34]:
import json

def contains(strings: list[str], substring) -> bool:
    for string in strings:
        if string != substring and substring in string:
            return False
    return True

with open("./all/openings.json", "r") as json_file:
    openings: list[dict[str]] = json.load(json_file)

all_pgn: list[str] = list(map(lambda opening: opening.get("pgn", ""), openings))

lessons_openings = list(filter(
    lambda opening: contains(all_pgn, opening.get("pgn", "")), openings
))

In [35]:
import json

with open("./lessons/openings.json", "w", encoding="utf-8") as lessons_openings_string_json_file:
    json.dump(lessons_openings, lessons_openings_string_json_file, ensure_ascii=False, indent=2)

In [36]:
def get_moves(pgn: str) -> list[str]:
    items: list[str] = pgn.split(" ")
    moves: list[str] = [item for index, item in enumerate(items) if index % 3 != 0]
    return moves

In [37]:
def get_half_moves(pgn: str) -> int:
    moves = get_moves(pgn)
    return len(moves)

In [38]:
def get_main_line_score(pgn: str, pgn_list: list[str]):
    moves_list: list[list[str]] = [get_moves(pgn_item) for pgn_item in pgn_list]
    moves: list[str] = get_moves(pgn)
    index: int = 1
    flag = True
    while flag:
        moves_string: str = "-".join(moves[0:index])
        moves_contained = [
            "-".join(moves_item)
            for moves_item in moves_list
            if "-".join(moves_item).startswith(moves_string)
        ]
        if len(moves_contained) <= 1 or len(moves) == index:
            flag = False
        else:
            index += 1
    print(pgn, index)
    return index

In [39]:
import chess.pgn
import io


def get_fen(pgn: str) -> str:
    pgn_string = io.StringIO(pgn)
    game = chess.pgn.read_game(pgn_string)
    board = game.board()
    for move in game.mainline_moves():
        board.push(move)
    return board.fen()

In [40]:
from stockfish import Stockfish

stockfish = Stockfish()


def get_evaluation(fen: str) -> float:
    stockfish.set_fen_position(fen)
    evaluation: dict = stockfish.get_evaluation()
    centipawn = evaluation.get("value", 0)
    pawn = centipawn / 100
    return pawn

In [41]:
import pandas

def get_value_or_empty(lst, index):
    """Returns the value at the given index in lst or an empty string if out of bounds."""
    return lst[index] if 0 <= index < len(lst) else ""

lessons_openings_data_frame = pandas.DataFrame(lessons_openings)
pgn_list: list[str] = lessons_openings_data_frame['pgn'].tolist()
lessons_openings_data_frame["first"] = lessons_openings_data_frame["pgn"].apply(lambda pgn: f"{get_value_or_empty(pgn.split(" "), 1)}-{get_value_or_empty(pgn.split(" "), 2)}")
lessons_openings_data_frame["half_moves"] = lessons_openings_data_frame["pgn"].apply(lambda pgn: get_half_moves(pgn))
lessons_openings_data_frame["fen"] = lessons_openings_data_frame["pgn"].apply(lambda pgn: get_fen(pgn))
lessons_openings_data_frame["evaluation"] = lessons_openings_data_frame["fen"].apply(lambda fen: get_evaluation(fen))
lessons_openings_data_frame["advantage"] = lessons_openings_data_frame["evaluation"].apply(lambda evaluation: "white" if evaluation > 0 else "black")
lessons_openings_data_frame["main_line_score"] = lessons_openings_data_frame["pgn"].apply(lambda pgn: get_main_line_score(pgn, pgn_list))
lessons_openings_data_frame = lessons_openings_data_frame.rename(columns={"name": "opening_variation"})
lessons_openings_data_frame["opening"] = lessons_openings_data_frame["opening_variation"].apply(lambda variation: f"{variation.split(":")[0]}")
lessons_openings_data_frame = lessons_openings_data_frame.sort_values(by=['first', 'opening', 'pgn', 'main_line_score'])
lessons_openings_data_frame.to_csv("./lessons/openings.csv", index=False)

1. Nh3 d5 2. g3 e5 3. f4 Bxh3 4. Bxh3 exf4 5. O-O fxg3 6. hxg3 1
1. e3 e5 2. c4 d6 3. Nc3 Nc6 4. b3 Nf6 3
1. a3 a5 2. b4 2
1. f3 e5 2. g4 Qh4# 3
1. f3 d5 2. e4 g6 3. d4 dxe4 4. c3 2
1. f3 f5 2. e4 fxe4 3. Nc3 2
1. f3 e5 2. Kf2 3
1. h3 h5 2. g4 2
1. h3 d5 2. a3 e5 2
1. a3 e5 2. b3 d5 3. c3 Nf6 4. d3 Nc6 5. e3 Bd6 6. f3 O-O 7. g3 3
1. a3 e5 2. g3 d5 3. Bg2 Nf6 4. d3 Nc6 5. Nd2 Bd6 6. e3 O-O 7. h3 3
1. h3 e5 2. a3 2
1. g4 f5 2
1. g4 g5 2. f4 2
1. g4 d5 2. Bg2 h5 3. gxh5 4
1. g4 d5 2. Bg2 Bxg4 3. c4 d4 4. Bxb7 Nd7 5. Bxa8 Qxa8 4
1. g4 d5 2. Bg2 e5 3. d4 exd4 4. c3 4
1. g4 d5 2. Bg2 c6 3. c4 dxc4 4. b3 5
1. g4 d5 2. h3 e5 3. Bg2 c6 4. d4 e4 5. c4 Bd6 6. Nc3 Ne7 3
1. g4 e5 2. h3 Nc6 3
1. g4 d5 2. Bg2 c6 3. g5 5
1. g4 e5 2. Bg2 d5 3. c4 3
1. g4 d5 2. e4 dxe4 3. Nc3 h5 6
1. g4 d5 2. e4 dxe4 3. Nc3 e5 4. d3 6
1. g3 Nc6 2. Nc3 d5 3. d4 e5 4. dxe5 d4 5. Ne4 f5 2
1. g3 e5 2. a3 d5 3. Nf3 e4 4. Nh4 Be7 5. d3 3
1. g3 d5 2. Bg2 e5 3. b4 5
1. g3 d5 2. Bg2 e6 4
1. g3 Nf6 2
1. g3 g5 2
1. g3 f5 2. e4 fxe

In [42]:
openings_group_counts = lessons_openings_data_frame[["first", "opening"]].value_counts()
openings_group_counts

first   opening               
e4-c5   Sicilian Defense          237
e4-e5   Ruy Lopez                 150
e4-e6   French Defense            114
e4-e5   Italian Game              101
        King's Gambit Accepted     92
                                 ... 
d4-Nf6  Canard Opening              1
        Döry Defense                1
        Marienbad System            1
        Mexican Defense             1
h4-g5   Kádas Opening               1
Name: count, Length: 254, dtype: int64

In [43]:
from re import sub


def snake_case(s):
    return "_".join(
        sub(
            "([A-Z][a-z]+)", r" \1", sub("([A-Z]+)", r" \1", s.replace("-", " "))
        ).split()
    ).lower()

In [44]:
from chess_gif.gif_maker import GIFMaker
import pathlib
from tqdm.notebook import tqdm

for index in tqdm(lessons_openings_data_frame.index):
    pgn: str = lessons_openings_data_frame["pgn"][index]
    first: str = lessons_openings_data_frame["first"][index]
    opening: str = lessons_openings_data_frame["opening"][index]
    half_moves: int = lessons_openings_data_frame["half_moves"][index]
    main_line_score: int = lessons_openings_data_frame["main_line_score"][index]
    opening_folder = (
        snake_case(opening).replace(":", "").replace(",", "").replace("'", "_")
    )
    opening_variation: str = lessons_openings_data_frame["opening_variation"][index]
    file_name = (
        snake_case(opening_variation)
        .replace(":", "")
        .replace(",", "")
        .replace("'", "_")
    )
    gif_maker = GIFMaker(delay=500, h_margin=0, v_margin=0)
    pgn_bytes = str.encode(pgn)
    pgn_folder = f"./lessons/pgn/{first}/{opening_folder}"
    pathlib.Path(pgn_folder).mkdir(parents=True, exist_ok=True)
    pgn_file = open(f"{pgn_folder}/{main_line_score}_{file_name}_{half_moves}.pgn", "w")
    print(f"{first}/{file_name}", pgn)
    pgn_file.write(pgn)
    pgn_file.close()
    gif_folder = f"./lessons/gif/{first}/{opening_folder}"
    pathlib.Path(gif_folder).mkdir(parents=True, exist_ok=True)
    gif_maker.make_gif_from_pgn_file(
        f"{pgn_folder}/{main_line_score}_{file_name}_{half_moves}.pgn",
        f"{gif_folder}/{main_line_score}_{file_name}_{half_moves}.gif",
    )

  0%|          | 0/2276 [00:00<?, ?it/s]

Na3-e5/sodium_attack_durkin_gambit 1. Na3 e5 2. Nc4 Nc6 3. e4 f5
Na3-e5/sodium_attack_celadon_variation 1. Na3 e5 2. d3 Bxa3 3. bxa3 d5 4. e3 c5 5. Rb1
Na3-g6/sodium_attack_chenoboskion_variation 1. Na3 g6 2. g4
Nc3-Nf6/van_geet_opening_tübingen_gambit 1. Nc3 Nf6 2. g4
Nc3-b5/van_geet_opening_laroche_gambit 1. Nc3 b5
Nc3-c5/van_geet_opening_sicilian_two_knights 1. Nc3 c5 2. Nf3 Nc6 3. d4 cxd4 4. Nxd4
Nc3-c5/van_geet_opening_twyble_attack 1. Nc3 c5 2. Rb1
Nc3-c5/van_geet_opening_düsseldorf_gambit 1. Nc3 c5 2. b4
Nc3-c5/van_geet_opening_novosibirsk_variation 1. Nc3 c5 2. d4 cxd4 3. Qxd4 Nc6 4. Qh4
Nc3-d5/van_geet_opening_venezolana_variation 1. Nc3 d5 2. d3 Nf6 3. g3
Nc3-d5/van_geet_opening_sleipnir_gambit 1. Nc3 d5 2. e3 e5 3. d4 Bb4
Nc3-d5/van_geet_opening_hector_gambit 1. Nc3 d5 2. e4 dxe4 3. Bc4
Nc3-d5/van_geet_opening_dunst_perrenet_gambit 1. Nc3 d5 2. e4 dxe4 3. d3
Nc3-d5/van_geet_opening_dougherty_gambit 1. Nc3 d5 2. e4 dxe4 3. f3
Nc3-d5/van_geet_opening_melleby_gambit 1. Nc3 d5 2

gifsicle: warning: trivial adaptive palette (only 253 colors in source)


a4-b5/ware_opening_wing_gambit 1. a4 b5 2. axb5 Bb7
a4-b6/ware_opening_cologne_gambit 1. a4 b6 2. d4 d5 3. Nc3 Nd7
a4-e5/ware_opening_meadow_hay_trap 1. a4 e5 2. Ra3
a4-e5/ware_opening_ware_gambit 1. a4 e5 2. a5 d5 3. e3 f5 4. a6


gifsicle: warning: trivial adaptive palette (only 253 colors in source)


...

gifsicle: warning: trivial adaptive palette (only 253 colors in source)
gifsicle: warning: trivial adaptive palette (only 253 colors in source)
gifsicle: warning: trivial adaptive palette (only 253 colors in source)


h4-d5/kádas_opening_kádas_gambit 1. h4 d5 2. d4 c5 3. Nf3 cxd4 4. c3
h4-d5/kádas_opening_myers_variation 1. h4 d5 2. d4 c5 3. e4
h4-e5/kádas_opening_kádas_gambit 1. h4 e5 2. d4 exd4 3. c3


gifsicle: warning: trivial adaptive palette (only 253 colors in source)


h4-f5/kádas_opening_steinbok_gambit 1. h4 f5 2. e4 fxe4 3. d3
h4-g5/kádas_opening_schneider_gambit 1. h4 g5


gifsicle: warning: trivial adaptive palette (only 253 colors in source)
